## 1. Project description:

Banks have a lot of information about one's credit history, monthly income and other material. From the previous cases, banks make mathematical based assumptions if person is capable of handling credit for money X which need to be repaid in Y years**

**Main task: predict probability if person who is willing to get a credit will be able to repay it.** 

* Based on Kaggle competition [Home Credit Default Risk](https://www.kaggle.com/c/home-credit-default-risk/data)
* Metric used: area under curve (AUC)
* Contains 2.5 GB of CSV files
* Expected Kaggle result: 0.786

CSV files connections

![](tree.png)

## 2. Project.

Basic imports

In [1]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")
%load_ext nb_black

<IPython.core.display.Javascript object>

Importing data

In [2]:
train_df = pd.read_csv("application_train.csv")

<IPython.core.display.Javascript object>

In [3]:
test_df = pd.read_csv("application_test.csv")

<IPython.core.display.Javascript object>

In [4]:
bureau_df = pd.read_csv("bureau.csv")

<IPython.core.display.Javascript object>

In [5]:
bureau_balance_df = pd.read_csv("bureau_balance.csv")

<IPython.core.display.Javascript object>

In [6]:
previous_application_df = pd.read_csv("previous_application.csv")

<IPython.core.display.Javascript object>

In [7]:
pos_cash_balance_df = pd.read_csv("POS_CASH_balance.csv")

<IPython.core.display.Javascript object>

In [8]:
instalments_payments_df = pd.read_csv("installments_payments.csv")

<IPython.core.display.Javascript object>

In [9]:
credit_card_balance_df = pd.read_csv("credit_card_balance.csv")

<IPython.core.display.Javascript object>

### 2-1. Before EDA, data from supporting CSVs have to be represented in train_df and test_df. It could be achieved by digging deeper into the connections and extracting only relevant information.

**2-1-1.** Exploring credit bureau branch (bureau_df and bureau_balance_df). First, bureau_df needs to be filtered based on id which exist in train_df or test_df.

In [10]:
# extracting values as list from train_df
relevant_ids = train_df["SK_ID_CURR"].values.tolist()

# extending list with values from test_df
relevant_ids.extend(test_df["SK_ID_CURR"].values.tolist())

<IPython.core.display.Javascript object>

In [11]:
# filtering dataframe
bureau_df = bureau_df[bureau_df["SK_ID_CURR"].isin(relevant_ids)]

<IPython.core.display.Javascript object>

Next step, let's filter bureau_balance_df on credit_id which are revelant.

In [12]:
# extracting relevant credit ids
relevant_ids = bureau_df["SK_ID_BUREAU"].values.tolist()

<IPython.core.display.Javascript object>

In [13]:
# filtering dataframe
bureau_balance_df = bureau_balance_df[
    bureau_balance_df["SK_ID_BUREAU"].isin(relevant_ids)
]

<IPython.core.display.Javascript object>

In [14]:
bureau_balance_df.groupby(["SK_ID_BUREAU", "STATUS"]).count()

MONTHS_BALANCE
SK_ID_BUREAU STATUS                
5001710      0                    5
             C                   48
             X                   30
5001711      0                    3
             X                    1
...                             ...
6842887      0                    6
             C                   31
6842888      0                    9
             1                    1
             C                   52

[1631098 rows x 1 columns]

<IPython.core.display.Javascript object>

Here: SK_ID_BUREAU stands for credit ID, there are 7 different statuses:
* C - completed credit repay
* X - no information
* 0 - payment done according to the plan
* 1 - payment done with up to 1 month delay
* 2 - payment done with up to 2 month delay
* 3 - payment done with up to 3 month delay
* 4 - payment done with up to 4 month delay
* 5 - payment done with longer delay than 4 months

Next step will be exporting counts to bureau_df based on credit id as new column.

In [15]:
# creating new dataframe which could be afterwards merged with bureau_df

extracted_df = bureau_balance_df.groupby(["SK_ID_BUREAU"]).count().reset_index()

<IPython.core.display.Javascript object>

In [16]:
# dropping irrelevant col
extracted_df = extracted_df.drop(["MONTHS_BALANCE"], axis=1)

<IPython.core.display.Javascript object>

In [17]:
# renaming cols
extracted_df.columns = ["SK_ID_BUREAU", "COUNT"]

<IPython.core.display.Javascript object>

Now it's time to extract count for every single status per id.

In [18]:
# extracting status counts
status_counts_df = (
    bureau_balance_df.groupby(["SK_ID_BUREAU", "STATUS"]).count().reset_index()
)

<IPython.core.display.Javascript object>

In [19]:
# making pivot table of status counts
status_pivot_df = status_counts_df.pivot_table(
    "MONTHS_BALANCE", "SK_ID_BUREAU", "STATUS"
)

<IPython.core.display.Javascript object>

In [20]:
# filling NaN values with 0
status_pivot_df = status_pivot_df.fillna(0)

<IPython.core.display.Javascript object>

In [21]:
# merging two dataframes
extracted_df = pd.merge(
    extracted_df, status_pivot_df, left_on="SK_ID_BUREAU", right_on="SK_ID_BUREAU"
)

<IPython.core.display.Javascript object>

Last step before merging it with bureau_df is to calculate ratios (because ratios represent better than absolute numbers if a person is willing to pay the credit and understand the risks behind it). Status C will be treated as 0. Same situation with status X (in favor of applicant).

In [22]:
# calculating ratios
extracted_df["ratio_0"] = (
    extracted_df["0"] + extracted_df["C"] + extracted_df["X"]
) / extracted_df["COUNT"]

other_statuses = [str(x) for x in range(1, 6)]

for status in other_statuses:
    new_name = "ratio_" + status
    extracted_df[new_name] = extracted_df[status] / extracted_df["COUNT"]

<IPython.core.display.Javascript object>

In [23]:
join_cols = [
    "SK_ID_BUREAU",
    "ratio_0",
    "ratio_1",
    "ratio_2",
    "ratio_3",
    "ratio_4",
    "ratio_5",
]
extracted_df[join_cols]

,SK_ID_BUREAU,ratio_0,ratio_1,ratio_2,ratio_3,ratio_4,ratio_5
0,5001710,1.000000,0.000000,0.0,0.0,0.0,0.0
1,5001711,1.000000,0.000000,0.0,0.0,0.0,0.0
2,5001712,1.000000,0.000000,0.0,0.0,0.0,0.0
3,5001713,1.000000,0.000000,0.0,0.0,0.0,0.0
4,5001714,1.000000,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
774349,6842884,1.000000,0.000000,0.0,0.0,0.0,0.0
774350,6842885,0.500000,0.000000,0.0,0.0,0.0,0.5
774351,6842886,1.000000,0.000000,0.0,0.0,0.0,0.0
774352,6842887,1.000000,0.000000,0.0,0.0,0.0,0.0


<IPython.core.display.Javascript object>

In [24]:
# merging bureau_balance_df main insights to bureau_df
bureau_df = pd.merge(bureau_df, extracted_df[join_cols], on="SK_ID_BUREAU", how="left")

<IPython.core.display.Javascript object>

**2-1-2**. Exploring bureau_df and finding the best features that could be exported to train and test dfs.

In [25]:
bureau_df

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,...,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,ratio_0,ratio_1,ratio_2,ratio_3,ratio_4,ratio_5
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,...,0.0,Consumer credit,-131,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,...,0.0,Credit card,-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,...,0.0,Consumer credit,-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,...,0.0,Credit card,-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,...,0.0,Consumer credit,-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,259355,5057750,Active,currency 1,-44,0,-30.0,NaN,0.0,0,...,0.0,Microloan,-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1716424,100044,5057754,Closed,currency 1,-2648,0,-2433.0,-2493.0,5476.5,0,...,0.0,Consumer credit,-2493,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1716425,100044,5057762,Closed,currency 1,-1809,0,-1628.0,-970.0,NaN,0,...,0.0,Consumer credit,-967,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1716426,246829,5057770,Closed,currency 1,-1878,0,-1513.0,-1513.0,NaN,0,...,0.0,Consumer credit,-1508,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<IPython.core.display.Javascript object>

In [26]:
# creating df for exporting to train_df and test_df
export_df = pd.DataFrame()
export_df["SK_ID_CURR"] = bureau_df.SK_ID_CURR.unique()

<IPython.core.display.Javascript object>

Column credit active could be transformed into pivot table and added to export_df as experience about applicant, on how many credits did he take and how many of them are still active.

In [27]:
# extracting status counts
status_counts_df = (
    bureau_df.groupby(["SK_ID_CURR", "CREDIT_ACTIVE"]).count().reset_index()
)

<IPython.core.display.Javascript object>

In [28]:
# making pivot table of status counts
status_pivot_df = status_counts_df.pivot_table(
    "CREDIT_CURRENCY", "SK_ID_CURR", "CREDIT_ACTIVE"
)
# filling NaN values with 0
status_pivot_df = status_pivot_df.fillna(0)

<IPython.core.display.Javascript object>

In [29]:
# merging pivot table with export_df
export_df = pd.merge(
    export_df, status_pivot_df, left_on="SK_ID_CURR", right_on="SK_ID_CURR", how="left"
)

<IPython.core.display.Javascript object>

Next, adding how many times person took credit in particular currency.

In [30]:
# extracting currency counts
currency_counts_df = (
    bureau_df.groupby(["SK_ID_CURR", "CREDIT_CURRENCY"]).count().reset_index()
)

<IPython.core.display.Javascript object>

In [31]:
# making pivot table of status counts
currency_pivot_df = currency_counts_df.pivot_table(
    "CREDIT_ACTIVE", "SK_ID_CURR", "CREDIT_CURRENCY"
)
# filling NaN values with 0
currency_pivot_df = currency_pivot_df.fillna(0)

<IPython.core.display.Javascript object>

In [32]:
# merging pivot table with export_df
export_df = pd.merge(
    export_df,
    currency_pivot_df,
    left_on="SK_ID_CURR",
    right_on="SK_ID_CURR",
    how="left",
)

<IPython.core.display.Javascript object>

Most of what is left are numerical, all they will be calculated as averages and added for every applicant.

In [33]:
avg_list = [
    "DAYS_CREDIT",
    "CREDIT_DAY_OVERDUE",
    "DAYS_CREDIT_ENDDATE",
    "DAYS_ENDDATE_FACT",
    "AMT_CREDIT_MAX_OVERDUE",
    "CNT_CREDIT_PROLONG",
    "AMT_CREDIT_SUM",
    "AMT_CREDIT_SUM_DEBT",
    "AMT_CREDIT_SUM_LIMIT",
    "AMT_CREDIT_SUM_OVERDUE",
    "DAYS_CREDIT_UPDATE",
    "AMT_ANNUITY",
    "ratio_0",
    "ratio_1",
    "ratio_2",
    "ratio_3",
    "ratio_4",
    "ratio_5",
]

# extracting averages
averages_df = bureau_df.groupby(["SK_ID_CURR"])[avg_list].mean()

<IPython.core.display.Javascript object>

In [34]:
# putting feature into export_df
export_df = pd.merge(
    export_df,
    averages_df,
    left_on="SK_ID_CURR",
    right_on="SK_ID_CURR",
    how="left",
)

<IPython.core.display.Javascript object>

Last one left is categorical, for what reason credit is taken. Let's try pivot table approach.

In [35]:
# extracting type counts
type_counts_df = bureau_df.groupby(["SK_ID_CURR", "CREDIT_TYPE"]).count().reset_index()

<IPython.core.display.Javascript object>

In [36]:
# making pivot table of status counts
type_pivot_df = type_counts_df.pivot_table("CREDIT_ACTIVE", "SK_ID_CURR", "CREDIT_TYPE")
# filling NaN values with 0
type_pivot_df = type_pivot_df.fillna(0)

<IPython.core.display.Javascript object>

In [37]:
# merging pivot table with export_df
export_df = pd.merge(
    export_df,
    type_pivot_df,
    left_on="SK_ID_CURR",
    right_on="SK_ID_CURR",
    how="left",
)

<IPython.core.display.Javascript object>

In [38]:
# filling left missing values with 0 (in favor for applicant)
export_df = export_df.fillna(0)

<IPython.core.display.Javascript object>

In [39]:
# renaming cols
export_df.columns = [
    "SK_ID_CURR",
    "STAT_ACTIVE",
    "STAT_BAD_DEPT",
    "STAT_CLOSED",
    "STAT_SOLD",
    "CURR_1",
    "CURR_2",
    "CURR_3",
    "CURR_4",
    "DAYS_CREDIT",
    "CREDIT_DAY_OVERDUE",
    "DAYS_CREDIT_ENDDATE",
    "DAYS_ENDDATE_FACT",
    "AMT_CREDIT_MAX_OVERDUE",
    "CNT_CREDIT_PROLONG",
    "AMT_CREDIT_SUM",
    "AMT_CREDIT_SUM_DEBT",
    "AMT_CREDIT_SUM_LIMIT",
    "AMT_CREDIT_SUM_OVERDUE",
    "DAYS_CREDIT_UPDATE",
    "AMT_ANNUITY",
    "OVERDUE_RATIO_0",
    "OVERDUE_RATIO_1",
    "OVERDUE_RATIO_2",
    "OVERDUE_RATIO_3",
    "OVERDUE_RATIO_4",
    "OVERDUE_RATIO_5",
    "TYPE_OTHER",
    "TYPE_CAR",
    "TYPE_CASH",
    "TYPE_CONSUMER",
    "TYPE_CREDIT_CARD",
    "TYPE_REPAY",
    "TYPE_BUSINESS",
    "TYPE_SHARES",
    "TYPE_EQUIPMENT",
    "TYPE_REPLENISHING",
    "TYPE_MICRO",
    "TYPE_OPERATOR",
    "TYPE_MORTGAGE",
    "TYPE_REAL_ESTATE",
    "TYPE_UNKNOWN",
]

<IPython.core.display.Javascript object>

In [40]:
# merging with train_df and test_df
train_df = pd.merge(
    train_df,
    export_df,
    left_on="SK_ID_CURR",
    right_on="SK_ID_CURR",
    how="left",
)

test_df = pd.merge(
    test_df,
    export_df,
    left_on="SK_ID_CURR",
    right_on="SK_ID_CURR",
    how="left",
)

<IPython.core.display.Javascript object>

**2-1-3**. Digging into POS_CASH_balance.csv and its relations to previous_applications.csv. Relevant things will be exported to previous_applications df.

In [41]:
pos_cash_balance_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001358 entries, 0 to 10001357
Data columns (total 8 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   SK_ID_PREV             int64  
 1   SK_ID_CURR             int64  
 2   MONTHS_BALANCE         int64  
 3   CNT_INSTALMENT         float64
 4   CNT_INSTALMENT_FUTURE  float64
 5   NAME_CONTRACT_STATUS   object 
 6   SK_DPD                 int64  
 7   SK_DPD_DEF             int64  
dtypes: float64(2), int64(5), object(1)
memory usage: 610.4+ MB


<IPython.core.display.Javascript object>

For all numerical features, we will extract averages.

In [42]:
avg_list = [
    "MONTHS_BALANCE",
    "CNT_INSTALMENT",
    "CNT_INSTALMENT_FUTURE",
    "SK_DPD",
    "SK_DPD_DEF",
]

# extracting averages
averages_df = pos_cash_balance_df.groupby(["SK_ID_PREV"])[avg_list].mean()

<IPython.core.display.Javascript object>

In [43]:
# creating export df
export_df = pd.DataFrame()

export_df["SK_ID_PREV"] = previous_application_df.SK_ID_PREV.unique()

# assigning averages to export df
export_df = pd.merge(
    export_df,
    averages_df,
    left_on="SK_ID_PREV",
    right_on="SK_ID_PREV",
    how="left",
)

<IPython.core.display.Javascript object>

Categorical feature NAME_CONTRACT_STATUS was already covered in the previous csv file, so will not be added to export_df.

In [44]:
# merging export_df with previous_application_df
previous_application_df = pd.merge(
    previous_application_df,
    export_df,
    left_on="SK_ID_PREV",
    right_on="SK_ID_PREV",
    how="left",
)

<IPython.core.display.Javascript object>

**2-1-4**. Exploring instalments_payments csv and exporting significant features to previous_application_df.

All features in instalments_payments_df are numerical, so averages will be extracted and inserted into previous_application_df.

In [45]:
avg_list = [
    "NUM_INSTALMENT_VERSION",
    "NUM_INSTALMENT_NUMBER",
    "DAYS_INSTALMENT",
    "DAYS_ENTRY_PAYMENT",
    "AMT_INSTALMENT",
    "AMT_PAYMENT",
]

# extracting averages
averages_df = instalments_payments_df.groupby(["SK_ID_PREV"])[avg_list].mean()

<IPython.core.display.Javascript object>

In [46]:
# creating export df
export_df = pd.DataFrame()

export_df["SK_ID_PREV"] = previous_application_df.SK_ID_PREV.unique()

<IPython.core.display.Javascript object>

In [47]:
# assigning averages to export df
export_df = pd.merge(
    export_df,
    averages_df,
    left_on="SK_ID_PREV",
    right_on="SK_ID_PREV",
    how="left",
)

export_df = export_df.fillna(0)

<IPython.core.display.Javascript object>

In [48]:
# merging export_df with previous_application_df
previous_application_df = pd.merge(
    previous_application_df,
    export_df,
    left_on="SK_ID_PREV",
    right_on="SK_ID_PREV",
    how="left",
)

<IPython.core.display.Javascript object>

**2-1-5**. Exploring credit_card_balance csv and exporting significant features to previous_application_df.

In [49]:
credit_card_balance_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3840312 entries, 0 to 3840311
Data columns (total 23 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   SK_ID_PREV                  int64  
 1   SK_ID_CURR                  int64  
 2   MONTHS_BALANCE              int64  
 3   AMT_BALANCE                 float64
 4   AMT_CREDIT_LIMIT_ACTUAL     int64  
 5   AMT_DRAWINGS_ATM_CURRENT    float64
 6   AMT_DRAWINGS_CURRENT        float64
 7   AMT_DRAWINGS_OTHER_CURRENT  float64
 8   AMT_DRAWINGS_POS_CURRENT    float64
 9   AMT_INST_MIN_REGULARITY     float64
 10  AMT_PAYMENT_CURRENT         float64
 11  AMT_PAYMENT_TOTAL_CURRENT   float64
 12  AMT_RECEIVABLE_PRINCIPAL    float64
 13  AMT_RECIVABLE               float64
 14  AMT_TOTAL_RECEIVABLE        float64
 15  CNT_DRAWINGS_ATM_CURRENT    float64
 16  CNT_DRAWINGS_CURRENT        int64  
 17  CNT_DRAWINGS_OTHER_CURRENT  float64
 18  CNT_DRAWINGS_POS_CURRENT    float64
 19  CNT_INSTALMENT_MATURE

<IPython.core.display.Javascript object>

Most of credit_card_balance features are numerical, averages will be extracted from them.

In [50]:
avg_list = [
    "MONTHS_BALANCE",
    "AMT_BALANCE",
    "AMT_CREDIT_LIMIT_ACTUAL",
    "AMT_DRAWINGS_ATM_CURRENT",
    "AMT_DRAWINGS_CURRENT",
    "AMT_DRAWINGS_OTHER_CURRENT",
    "AMT_DRAWINGS_POS_CURRENT",
    "AMT_INST_MIN_REGULARITY",
    "AMT_PAYMENT_CURRENT",
    "AMT_PAYMENT_TOTAL_CURRENT",
    "AMT_RECEIVABLE_PRINCIPAL",
    "AMT_RECIVABLE",
    "AMT_TOTAL_RECEIVABLE",
    "CNT_DRAWINGS_ATM_CURRENT",
    "CNT_DRAWINGS_CURRENT",
    "CNT_DRAWINGS_OTHER_CURRENT",
    "CNT_DRAWINGS_POS_CURRENT",
    "CNT_INSTALMENT_MATURE_CUM",
    "SK_DPD",
    "SK_DPD_DEF",
]

# extracting averages
averages_df = credit_card_balance_df.groupby(["SK_ID_PREV"])[avg_list].mean()

<IPython.core.display.Javascript object>

In [51]:
# creating export df
export_df = pd.DataFrame()

export_df["SK_ID_PREV"] = previous_application_df.SK_ID_PREV.unique()

<IPython.core.display.Javascript object>

In [52]:
# assigning averages to export df
export_df = pd.merge(
    export_df,
    averages_df,
    left_on="SK_ID_PREV",
    right_on="SK_ID_PREV",
    how="left",
)

<IPython.core.display.Javascript object>

Extracting categorical feature NAME_CONTRACT_STATUS with pivot table.

In [53]:
# extracting status counts
status_counts_df = (
    credit_card_balance_df.groupby(["SK_ID_PREV", "NAME_CONTRACT_STATUS"])
    .count()
    .reset_index()
)

<IPython.core.display.Javascript object>

In [54]:
# making pivot table of status counts
status_pivot_df = status_counts_df.pivot_table(
    "MONTHS_BALANCE", "SK_ID_PREV", "NAME_CONTRACT_STATUS"
)

<IPython.core.display.Javascript object>

In [55]:
# assigning pivot table to export df
export_df = pd.merge(
    export_df,
    status_pivot_df,
    left_on="SK_ID_PREV",
    right_on="SK_ID_PREV",
    how="left",
)

<IPython.core.display.Javascript object>

In [56]:
export_df = export_df.fillna(0)

<IPython.core.display.Javascript object>

In [57]:
# merging export_df with previous_application_df
previous_application_df = pd.merge(
    previous_application_df,
    export_df,
    left_on="SK_ID_PREV",
    right_on="SK_ID_PREV",
    how="left",
)

<IPython.core.display.Javascript object>

**2-1-6**. Exploring previous_application_df and extract relevant features. Based on SK_ID_CURR these features will be assigned to train_df and test_df.

In [58]:
previous_application_df.head(5)

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,CNT_INSTALMENT_MATURE_CUM,SK_DPD_y,SK_DPD_DEF_y,Active,Approved,Completed,Demand,Refused,Sent proposal,Signed
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<IPython.core.display.Javascript object>

Categorical features from this database already exist in train ant test df. Only numerical features will be exported.

In [59]:
# filling missing values
previous_application_df = previous_application_df.fillna(0)

<IPython.core.display.Javascript object>

In [60]:
# extracting average list
avg_list = previous_application_df.select_dtypes(include=np.number).iloc[:, 2:].columns

<IPython.core.display.Javascript object>

In [61]:
# extracting averages
averages_df = previous_application_df.groupby(["SK_ID_CURR"])[avg_list].mean()

<IPython.core.display.Javascript object>

In [62]:
# creating export df
export_df = pd.DataFrame()

export_df["SK_ID_CURR"] = previous_application_df.SK_ID_CURR.unique()

<IPython.core.display.Javascript object>

In [63]:
# assigning averages to export df
export_df = pd.merge(
    export_df,
    averages_df,
    left_on="SK_ID_CURR",
    right_on="SK_ID_CURR",
    how="left",
)

<IPython.core.display.Javascript object>

Inserting previous applications count.

In [64]:
# extracting counts
counts_df = (
    previous_application_df.groupby("SK_ID_CURR").count().reset_index().iloc[:, :2]
)

<IPython.core.display.Javascript object>

In [65]:
# renaming columns
counts_df.columns = ["SK_ID_CURR", "PREV_COUNTS"]

<IPython.core.display.Javascript object>

In [66]:
# assigning counts to export df
export_df = pd.merge(
    export_df,
    counts_df,
    left_on="SK_ID_CURR",
    right_on="SK_ID_CURR",
    how="left",
)

# filling missing values
export_df = export_df.fillna(0)

<IPython.core.display.Javascript object>

In [67]:
# merging with train_df and test_df
train_df = pd.merge(
    train_df,
    export_df,
    left_on="SK_ID_CURR",
    right_on="SK_ID_CURR",
    how="left",
)

test_df = pd.merge(
    test_df,
    export_df,
    left_on="SK_ID_CURR",
    right_on="SK_ID_CURR",
    how="left",
)

<IPython.core.display.Javascript object>

**2-1-8** Saving merged train and test dataframes.

In [68]:
train_df.to_csv("merged_train.csv", index=False)
test_df.to_csv("merged_test.csv", index=False)

<IPython.core.display.Javascript object>